In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from DarcySolver import solve_darcy_on_doughnut_brain

T = 2.0           # final time
num_steps = 50    # number of time steps
dt = T / num_steps

resolutions = [10, 50, 100]    # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

kappas = [1e-8, 1e-10, 1e-12, 1e-14, 1e-16, 1e-18]
visc = 0.8*1e-3     # viscocity 
K_values = [kappa/visc for kappa in kappas]      # hydraulic conductivity

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)
c = 7 
mmHg2Pa = 133.32


In [ ]:
# create mesh
meshes = []
for N in resolutions:
    mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))
    meshes.append(mesh)

probe_points = [
                Point(ventricle_radius + 0.2*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.4*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.6*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.8*(brain_radius - ventricle_radius), 0)
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]

p_obs = Expression("""c*mmHg2Pa*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))
                   *( sqrt( x[0]*x[0] + x[1]*x[1] ) - R_ventricle ) """,
                    a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,c=c,
                    mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                    degree = 2)
p_skull = p_obs
p_ventricle = p_obs

f = Constant(0.0)

solutions = []
for i, kappa in enumerate(kappas):
    K = kappa/visc
    solutions.append([])
    for mesh in meshes:
        result = solve_darcy_on_doughnut_brain(mesh, f, T, num_steps, K=K,
                                                probe_points=probe_points,
                                                slice_points=slice_points,
                                                dirichlet_boundary_skull=p_skull,
                                                dirichlet_boundary_ventricle=p_ventricle)
        solutions[i].append(result)

In [ ]:
times = [0.4, 0.8, 1.2, 1.6, 2.0]

idx = []
for i in range(num_steps):
    if np.isclose((i + 1)*dt, times).any():
        idx.append(i)

for i, kappa in enumerate(kappas):
    K = kappa/visc
    fig, axes = plt.subplots(nrows=1, ncols=len(meshes), figsize=(15,6))
    for j, res in enumerate(resolutions):
        cross_section = solutions[i][j]["slice"]
        plt.subplot(1,len(resolutions),j + 1)
        for n in idx:
            plt.plot(x_coords, cross_section[n,:]/mmHg2Pa,
                     label="t = {}".format((n +1)*dt))
        plt.legend()
        plt.grid()
        plt.xlabel("x in m")
        plt.ylabel("p in mmHg")
        #plt.title(f"kappa={kappa:.1e},   h_min = {mesh.hmin():.2e}")
        plt.title(f"kappa={kappa:.1e},   N = {res}")


In [ ]:
# time refinement
N = 50
mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))

num_time_steps = [20, 50, 100]
time_ref_solutions = []
for i, kappa in enumerate(kappas):
    K = kappa/visc
    time_ref_solutions.append([])
    for num_steps in num_time_steps:
        result = solve_darcy_on_doughnut_brain(mesh, f, T, num_steps, K=K,
                                                probe_points=probe_points,
                                                slice_points=slice_points,
                                                dirichlet_boundary_skull=p_skull,
                                                dirichlet_boundary_ventricle=p_ventricle)
        time_ref_solutions[i].append(result)

In [ ]:
times = [0.4, 0.8, 1.2, 1.6, 2.0]

idx = []
for i in range(num_steps):
    if np.isclose((i + 1)*dt, times).any():
        idx.append(i)

for i, kappa in enumerate(kappas):
    K = kappa/visc
    fig, axes = plt.subplots(nrows=1, ncols=len(meshes), figsize=(15,6))
    for j, num_steps in enumerate(time_ref_solutions):
        cross_section = solutions[i][j]["slice"]
        plt.subplot(1,len(time_ref_solutions),j + 1)
        for n in idx:
            plt.plot(x_coords, cross_section[n,:]/mmHg2Pa,
                     label="t = {}".format((n +1)*dt))
        plt.legend()
        plt.grid()
        plt.xlabel("x in m")
        plt.ylabel("p in mmHg")
        #plt.title(f"kappa={kappa:.1e},   h_min = {mesh.hmin():.2e}")
        plt.title(f"kappa={kappa:.1e},   dt = {T/num_steps:%.4f}")